In [77]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import psycopg2 as pg   
from pyspark.sql import SparkSession 
from pyspark.sql import Window
import sys
from pyspark.sql.functions import sum, bround
from pyspark.sql import DataFrameReader
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,count
import pandas as pd
from pyspark.sql import functions as f
from pyspark.sql.functions import desc


In [2]:
connection = pg.connect(user = "postgres",password = "7980",host = "127.0.0.1",port = "5432",database = "Data_warehouse")

In [3]:
connection.cursor()

<cursor object at 0x000002495D4D1BA8; closed: 0>

In [4]:
spark = SparkSession \
.builder \
.appName("Python_Spark_SQL for Data_warehouse Project") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()


In [250]:
donations=pd.read_sql_query("Select * from donations limit 200000" ,con=connection)
team=pd.read_sql_query("Select * from teams limit 200000", con=connection)
participants=pd.read_sql_query("select * from participants limit 200000 ", con=connection)
avgyear=pd.read_sql_query("select * from avgyear limit 200000", con=connection)
city=pd.read_sql_query("select * from city limit 100000", con=connection)
donor=pd.read_sql_query("select * from donor limit 100000", con=connection)
gender=pd.read_sql_query("select * from gender limit 100000", con=connection)
location=pd.read_sql_query("select * from location limit 100000", con=connection)

events=pd.read_sql_query("select * from events limit 100000", con=connection)
teams1=pd.read_sql_query("select * from teams1 limit 200000", con=connection)
participants1=pd.read_sql_query("select * from participants1 limit 200000", con=connection)

In [251]:

team_df = spark.createDataFrame(team)

donations_df = spark.createDataFrame(donations)

participants_df = spark.createDataFrame(participants) 

avgyear_df = spark.createDataFrame(avgyear)

city_df = spark.createDataFrame(city)
donor_df = spark.createDataFrame(donor)
gender_df = spark.createDataFrame(gender)
location_df = spark.createDataFrame(location)

events_df = spark.createDataFrame(events)
teams1_df = spark.createDataFrame(teams1)
participants1_df = spark.createDataFrame(participants1)




In [45]:
#Q1. What is the total amount of givings  every team(is prior participants)did per year
team_df.join(donations_df, on="team_id",how = "inner")\
.groupby(team_df.ispriorparticipant,donations_df.fiscal_year)\
.agg(f.sum(donations_df.gift_amount+donations_df.additional_gift_amount).alias("total_amount")) \
.show()

+------------------+-----------+------------+
|ispriorparticipant|fiscal_year|total_amount|
+------------------+-----------+------------+
|              true|       2014|     4064579|
|              true|       2015|     8526540|
+------------------+-----------+------------+



In [165]:

#Q2. What are the  cities and respective states where  is done the majority of givings?
location_df.groupby("city","state","total_amount")\
.agg(f.max("total_amount"))\
.orderBy('total_amount', ascending=False)\
.show(10)


+------------+-----+------------+-----------------+
|        city|state|total_amount|max(total_amount)|
+------------+-----+------------+-----------------+
|     Houston|   TX|    36900736|         36900736|
|     NewYork|   NY|     8437563|          8437563|
|   Princeton|   NJ|     8040216|          8040216|
|      Denver|   CO|     5402230|          5402230|
|      Dallas|   TX|     4350647|          4350647|
|  SanAntonio|   TX|     3980558|          3980558|
|      Austin|   TX|     3674821|          3674821|
|     Chicago|   IL|     3584394|          3584394|
| Minneapolis|   MN|     3497011|          3497011|
|SanFrancisco|   CA|     3454297|          3454297|
+------------+-----+------------+-----------------+
only showing top 10 rows



In [121]:
#Q3.What is the average amount of donations for a particular event in a given year? 
avgyear_df.groupby("event_id","fiscal_year")\
.agg(f.avg("total_amount"))\
.limit(300).show(10)



+--------+-----------+-----------------+
|event_id|fiscal_year|avg(total_amount)|
+--------+-----------+-----------------+
|   21466|       2013|         400085.0|
|   21238|       2013|         780090.0|
|   28197|       2017|        1272985.0|
|   28956|       2017|         232754.0|
|   29037|       2017|         200321.0|
|   21114|       2013|          51963.0|
|   23828|       2014|          40196.0|
|   23948|       2014|           1915.0|
|   24000|       2014|         367560.0|
|   27060|       2016|         272694.0|
+--------+-----------+-----------------+
only showing top 10 rows



In [136]:
#Q4.What is the max amount of givings and max average of givings  based on donor’s gender ? 

gender_df.groupby("gender")\
.agg(f.max("total_amount"),f.max("avg_amount"))\
.show(3)



+-------------+-----------------+---------------+
|       gender|max(total_amount)|max(avg_amount)|
+-------------+-----------------+---------------+
|       Female|         32401149|       16200574|
|not_avaliable|         20990534|       10495267|
|         Male|         44409824|       22204912|
+-------------+-----------------+---------------+



In [141]:
#Q5.What is the max amount of givings  and max average of givings based on donor’s gender per each year?
#select max(total_amount),max(avg_amount),gender , fiscal_year from gender  group by gender,fiscal_year ;
gender_df.groupby("gender","fiscal_year")\
.agg(f.max("total_amount"),f.max("avg_amount"))\
.orderBy('fiscal_year', ascending=True)\
.show(15)


+-------------+-----------+-----------------+---------------+
|       gender|fiscal_year|max(total_amount)|max(avg_amount)|
+-------------+-----------+-----------------+---------------+
|not_avaliable|       2013|         17059512|        8529756|
|       Female|       2013|         31218075|       15609037|
|         Male|       2013|         43238109|       21619054|
|         Male|       2014|         44409824|       22204912|
|not_avaliable|       2014|         18234643|        9117321|
|       Female|       2014|         32048503|       16024251|
|       Female|       2015|         32401149|       16200574|
|not_avaliable|       2015|         17613162|        8806581|
|         Male|       2015|         44349808|       22174904|
|       Female|       2016|         30016462|       15008231|
|not_avaliable|       2016|         20990534|       10495267|
|         Male|       2016|         40121128|       20060564|
|not_avaliable|       2017|         17423312|        8711656|
|       

In [160]:

#Q6.What is the total amount of givings every team did based on their name and number of participants

team_df.join(donations_df, on="team_id",how = "inner")\
.groupby(team_df.ispriorparticipant,team_df.number_of_participants,team_df.name)\
.agg(f.sum(donations_df.gift_amount+donations_df.additional_gift_amount).alias("total_amount")) \
.show()

+------------------+----------------------+--------------------+------------+
|ispriorparticipant|number_of_participants|                name|total_amount|
+------------------+----------------------+--------------------+------------+
|              true|                    12|               Spoke|          35|
|              true|                    21|        Cyclomaniacs|        1090|
|              true|                    23|           MissionMS|          50|
|              true|                     2|         LilasLeague|         380|
|              true|                     9|     theWellFitGroup|         100|
|              true|                    11|DontMSwithNordsonEFD|          70|
|              true|                    11|BiogenIdecRolling...|        7767|
|              true|                    10|         TeamFanboys|          35|
|              true|                   242|         TeamCalpine|        4285|
|              true|                    23|         TourdeFarce|

In [163]:
#7.What are the top 5 cities and respective states where  is done the majority of givings? 


location_df.groupby("city","state","total_amount")\
.agg(f.max("total_amount"))\
.orderBy('total_amount', ascending=False)\
.limit(5).show()


+---------+-----+------------+-----------------+
|     city|state|total_amount|max(total_amount)|
+---------+-----+------------+-----------------+
|  Houston|   TX|    36900736|         36900736|
|  NewYork|   NY|     8437563|          8437563|
|Princeton|   NJ|     8040216|          8040216|
|   Denver|   CO|     5402230|          5402230|
|   Dallas|   TX|     4350647|          4350647|
+---------+-----+------------+-----------------+



In [239]:
#8.What is the total amount of givings per each type of gift(by donors) ordered by year ? 
donations_df.join(donor_df, on="donor_id",how = "inner")\
.groupby(donations_df.fiscal_year,donor_df.gifttype)\
.agg(f.sum(donations_df.gift_amount+donations_df.additional_gift_amount).alias("total_amount")) \
.orderBy('fiscal_year', ascending=False)\
.show(2)

+-----------+--------+------------+
|fiscal_year|gifttype|total_amount|
+-----------+--------+------------+
|       2014|  online|     2171621|
|       2014| offline|     1103057|
+-----------+--------+------------+



In [277]:
#what is the total amount givings from all the events    per state 


events_df.select("average_team_size","total_from_participant","state")\
.groupBy("state")\
.agg(f.sum(events_df.total_from_participant).alias("total_amount")) \
.orderBy("state").show();

+-----+------------+
|state|total_amount|
+-----+------------+
|   AK|      122438|
|   AL|      346378|
|   AR|      152060|
|   AZ|      215025|
|   CA|     3505923|
|   CO|     2427041|
|   CT|      488021|
|   DC|      101970|
|   DE|      499532|
|   FL|     2003579|
|   GA|      525703|
|   HI|       37045|
|   IA|       90996|
|   ID|      171775|
|   IL|     2412265|
|   IN|      159924|
|   KS|      832031|
|   KY|      132697|
|   LA|      746543|
|   MA|     1988092|
+-----+------------+
only showing top 20 rows



In [307]:
#which occupupation has the highest givings  
participants1_df.groupBy("participant_occupation")\
.agg(f.sum(participants1_df.total_from_participant).alias("total_amount"))\
.orderBy('total_amount', ascending=False)\
.show()




+----------------------+------------+
|participant_occupation|total_amount|
+----------------------+------------+
|         not_avaliable|    14421885|
|            Healthcare|      958179|
|           Engineering|      760309|
|  Information Techn...|      508962|
|  Executive/Management|      482642|
|                 Sales|      455294|
|            Consulting|      409925|
|   Legal and Paralegal|      323326|
|  Education and Tra...|      301797|
|  Banking and Finan...|      298265|
|            Accounting|      294621|
|  Real Estate, Rent...|      173542|
|             Marketing|      169082|
|         Manufacturing|      146639|
|  Construction and ...|      142663|
|  Science and Biote...|      137825|
|             Insurance|      115247|
|            Government|      108627|
|      Retail/Wholesale|      101619|
|  Transportation an...|       94582|
+----------------------+------------+
only showing top 20 rows



In [316]:
#get the events and the amount which has same value in year 2017 and count the number of times the event gave same amount  

teams1_df.filter(col("fiscal_year").startswith("2014"))\
.rollup("fiscal_year","event_type", teams1_df.total_offline_confirmed_gifts).count()\
.where(col("event_type").isNotNull()).orderBy("fiscal_year","event_type").show()



+-----------+----------+-----------------------------+-----+
|fiscal_year|event_type|total_offline_confirmed_gifts|count|
+-----------+----------+-----------------------------+-----+
|       2014|      Bike|                           76|    3|
|       2014|      Bike|                         5435|    1|
|       2014|      Bike|                          219|    1|
|       2014|      Bike|                         2384|    1|
|       2014|      Bike|                         1504|    1|
|       2014|      Bike|                          545|    5|
|       2014|      Bike|                        13532|    1|
|       2014|      Bike|                        20986|    1|
|       2014|      Bike|                        31089|    1|
|       2014|      Bike|                          494|    1|
|       2014|      Bike|                          871|    1|
|       2014|      Bike|                         1330|    1|
|       2014|      Bike|                        18691|    1|
|       2014|      Bike|

In [331]:
win_spec = Window.partitionBy('participant_occupation').orderBy('participant_occupation').rowsBetween(-sys.maxsize, 0)
cum_sum = participants1_df.select("participant_occupation","total_from_participant").withColumn('cumsum', f.sum(participants1_df.total_from_participant).over(win_spec))
cum_sum.show();


+----------------------+----------------------+------+
|participant_occupation|total_from_participant|cumsum|
+----------------------+----------------------+------+
|  Restaurant and Fo...|                     0|     0|
|  Restaurant and Fo...|                   200|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                     0|   200|
|  Restaurant and Fo...|                   150|   350|
|  Restaurant and Fo...|                     0|   350|
|  Restaurant and Fo...|                    10|   360|
|  Restaurant and Fo...|                     0|   360|
|  Restaurant and Fo...|                     0|   360|
|  Restaurant and Fo...|                     0|   360|
|  Restaur